In [1]:
import pandas as pd
import numpy as np
#import transformers
#from transformers import pipeline
#from transformers import AutoTokenizer
import random as r
import spacy
import tensorflow as tf
import torch
from collections import Counter
from collections import defaultdict
import altair as alt
import math
pd.set_option('display.max_columns', None)


In [2]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd /content/drive/MyDrive/W266-final-project

/content/drive/.shortcut-targets-by-id/1sJ6Lr7w3mgZD4tEPMBy2-elafN6GPZCY/W266-final-project


## Find the distribution of POS in summaries

In [ ]:
base = '/content/drive/MyDrive/W266-final-project/data/BBC News Summary/News Articles/'
file = open(base + 'business' + '/' + '001' + '.txt')
txt = nlp(file.read())
print(txt)

NameError: ignored

In [ ]:
base = '/content/drive/MyDrive/W266-final-project/data/BBC News Summary/Summaries/'
file = open(base + 'business' + '/' + '001' + '.txt')
txt = nlp(file.read())
print(txt)

TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn.For the full-year, TimeWarner posted a profit of $3.36bn, up 27% from its 2003 performance, while revenues grew 6.4% to $42.09bn.Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.However, the company said AOL's underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues.Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.For 2005, TimeWarner is projecting operating earnings growth of around 5%, and also expects higher revenue and wider profit margins.It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters.Time Warner's fourth quarter profits were slightly better than analysts' expectations.


# Create a dataframe for the summaries and news articles to be stored in.
Note: We're generating POS for the training and test datasets since that is ~130 articles which were randomly selected. This should give us a general idea of POS distribution for summaries and articles

In [4]:
indicies_df = pd.read_csv('/content/drive/MyDrive/W266-final-project/indicies.csv')
indicies_df.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)
indicies_df

,business,entertainment,politics,sport,tech
0,384,29,273,131,276
1,315,97,208,110,329
2,267,117,192,502,51
3,25,358,186,268,160
4,112,349,190,64,73
...,...,...,...,...,...
95,123,266,350,19,246
96,225,264,221,458,293
97,372,246,35,419,295
98,35,295,12,214,216


In [5]:
def get_types(base='/content/drive/MyDrive/W266-final-project/data/BBC News Summary/', summ_type='Summaries/', indicies_df=indicies_df):
  '''
  takes in base location of file directory, and summary types
  returns DF indexed by article number in indicies_df
  '''
  nlp = spacy.load("en_core_web_sm")
  final_df = indicies_df.copy()
  for t in indicies_df.columns:
    #holds the text for each articles
    txt1=[] 
    pos1 = []
    dep1 = []
    stop1 = []
    for i in indicies_df[t]: #iterates through each article
      if i<10:
        i='00' + str(i)
      elif i<100:
        i='0'+str(i)
      else:
        i=str(i)
      file = open(base + summ_type + t + '/' + i + '.txt', 'r', errors='replace')
      txt = nlp(file.read())
      #count each indivual POS in the text
      pos_types=[]
      dep_types = []
      is_stop_lst = []
      for token in txt:  #takes the parts adds the strings to a list
        pos_types.append(token.pos_)
        dep_types.append(token.dep_)
        is_stop_lst.append(token.is_stop)
      #takes the lists of strings, adds them to a master list for each category
      pos1.append(pos_types) 
      dep1.append(dep_types)
      stop1.append(is_stop_lst)
      txt1.append(txt)
    #add to DF- each column- whats in the list
    final_df[summ_type + '-'+ 'text'] = txt1
    final_df[summ_type + '-' + t+'-POS'] = pos1
    final_df[summ_type + '-' + t+'-DEP'] = dep1
    final_df[summ_type + '-' + t+'-STOP'] = stop1
    # list( list(article parts))
  return final_df


In [6]:
df_article = get_types(summ_type='News Articles/')
#df_article

In [7]:
df = get_types()
#df

In [8]:
df_final = pd.merge(df, df_article, how='inner', on=['business', 'entertainment', 'politics', 'sport', 'tech'])
#df_final

In [ ]:
#df_final.to_csv('EDA_masking_df.csv')

In [9]:
#get the count for each POS, then plot them by type to see if there is a difference within each category
cols = ['Summaries/-business-POS', 'Summaries/-business-DEP',
       'Summaries/-business-STOP', 'Summaries/-entertainment-POS',
       'Summaries/-entertainment-DEP', 'Summaries/-entertainment-STOP',
       'Summaries/-politics-POS', 'Summaries/-politics-DEP',
       'Summaries/-politics-STOP', 'Summaries/-sport-POS',
       'Summaries/-sport-DEP', 'Summaries/-sport-STOP', 'Summaries/-tech-POS',
       'Summaries/-tech-DEP', 'Summaries/-tech-STOP', 'News Articles/-text',
       'News Articles/-business-POS', 'News Articles/-business-DEP',
       'News Articles/-business-STOP', 'News Articles/-entertainment-POS',
       'News Articles/-entertainment-DEP', 'News Articles/-entertainment-STOP',
       'News Articles/-politics-POS', 'News Articles/-politics-DEP',
       'News Articles/-politics-STOP', 'News Articles/-sport-POS',
       'News Articles/-sport-DEP', 'News Articles/-sport-STOP',
       'News Articles/-tech-POS', 'News Articles/-tech-DEP',
       'News Articles/-tech-STOP']

def def_value():
    return 0

counts_df = df_final.copy()

for c in cols:
  cnts = defaultdict(def_value)
  cnt = Counter()
  for i in range(len(counts_df[c])):
    lst = str(counts_df[c][i]).split(',')
    for j,l in enumerate(lst):
      if ']' in l:
        lst[j] = l[:-1]
      if '[' in l:
        lst[j] = ' ' + l[1:]
    dictionaryy = Counter(lst)
    counts_df[c][i] = dictionaryy
    for k in dictionaryy:
      cnts[k] += dictionaryy[k]
  for k in cnts:
    cnts[k] = cnts[k]/100
  counts_df[c+' averages'] = [cnts] * len(counts_df[c])




/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
counts_df

In [10]:
df = pd.DataFrame.from_dict(counts_df['Summaries/-business-POS averages'][0].items())
df2 = pd.DataFrame.from_dict(counts_df['News Articles/-business-POS averages'][0].items())

df = pd.merge(df, df2, how='inner', on=[0])
df.rename(columns={0: '0'}, inplace=True)

In [11]:
#can't save dictionaries in DF to CSV- will get turned into strings
df2 = pd.DataFrame.from_dict(counts_df['News Articles/-business-POS averages'][0].items())
df2.rename(columns={0: 'POS', 1:'Avg'}, inplace=True)
df2

,POS,Avg
0,'ADJ',24.26
1,'NOUN',77.34
2,'PUNCT',41.64
3,'ADP',36.74
4,'VERB',42.49
5,'SPACE',5.18
6,'PROPN',35.84
7,'AUX',15.57
8,'DET',35.61
9,'SYM',2.41


In [12]:
df = pd.DataFrame.from_dict(counts_df['Summaries/-business-POS averages'][0].items())
#df

# **Compare the summaries & articles for each category**

In [15]:
def create_comparison_viz(df, df2, category):
  #get the upper limit and round up to the nearest 10
  #max1 = int(math.ceil(df[1].max() / 10.0) * 10)
  #max2 = int(math.ceil(df2[1].max() / 10.0) * 10)
  #upper_limit = max(max1, max2)


  chart1 = alt.Chart(df).mark_bar().encode(
      x= alt.X('avg:Q', title="Average count for Summary", scale=alt.Scale(domain=[0, 140])),
      y = alt.Y("POS:O", title="POS", sort='-x')
  )

  chart2 = alt.Chart(df2).mark_bar().encode(
      x=alt.X('avg:Q', title="Average count for Article", scale=alt.Scale(domain=[0, 140])),
      y= alt.Y("POS:O", title="", sort='-x') 
  )

  chart = alt.hconcat(chart1 | chart2)
  chart = chart.properties(title=f"Distribution of Parts of Speech (POS) for " + category).configure_title(anchor='middle')
  return chart

In [16]:
df_test1 = pd.DataFrame.from_dict(counts_df['Summaries/-business-POS averages'][0].items()).rename(columns={0: 'POS', 1: 'avg'})
df_test2 = pd.DataFrame.from_dict(counts_df['News Articles/-business-POS averages'][0].items()).rename(columns={0: 'POS', 1: 'avg'})


create_comparison_viz(df_test1, df_test2, 'business')

alt.HConcatChart(...)

In [17]:
#double check this one
df = pd.DataFrame.from_dict(counts_df['Summaries/-entertainment-POS averages'][0].items()).rename(columns={0: 'POS', 1: 'avg'}).rename(columns={0: 'POS', 1: 'avg'})
df_2 = pd.DataFrame.from_dict(counts_df['News Articles/-entertainment-POS averages'][0].items()).rename(columns={0: 'POS', 1: 'avg'}).rename(columns={0: 'POS', 1: 'avg'})

create_comparison_viz(df, df_2, 'entertainment')

alt.HConcatChart(...)

In [18]:
df = pd.DataFrame.from_dict(counts_df['Summaries/-politics-POS averages'][0].items()).rename(columns={0: 'POS', 1: 'avg'})
df_2 = pd.DataFrame.from_dict(counts_df['News Articles/-politics-POS averages'][0].items()).rename(columns={0: 'POS', 1: 'avg'})

create_comparison_viz(df, df_2, 'politics')

alt.HConcatChart(...)

In [19]:
df = pd.DataFrame.from_dict(counts_df['Summaries/-sport-POS averages'][0].items()).rename(columns={0: 'POS', 1: 'avg'})
df_2 = pd.DataFrame.from_dict(counts_df['News Articles/-sport-POS averages'][0].items()).rename(columns={0: 'POS', 1: 'avg'})

create_comparison_viz(df, df_2, 'sport')

alt.HConcatChart(...)

In [20]:
df = pd.DataFrame.from_dict(counts_df['Summaries/-tech-POS averages'][0].items()).rename(columns={0: 'POS', 1: 'avg'})
df_2 = pd.DataFrame.from_dict(counts_df['News Articles/-tech-POS averages'][0].items()).rename(columns={0: 'POS', 1: 'avg'})

create_comparison_viz(df, df_2, 'tech')

alt.HConcatChart(...)

# **Compare each category of summary to seeif there are any major differences**

In [32]:
df = pd.DataFrame.from_dict(counts_df['Summaries/-tech-POS averages'][0].items())
df2 = pd.DataFrame.from_dict(counts_df['Summaries/-sport-POS averages'][0].items())
df2.loc[len(df.index)] = ['X', 0.0]
df3 = pd.DataFrame.from_dict(counts_df['Summaries/-politics-POS averages'][0].items())
df4 = pd.DataFrame.from_dict(counts_df['Summaries/-entertainment-POS averages'][0].items())
df5 = pd.DataFrame.from_dict(counts_df['Summaries/-business-POS averages'][0].items())


def get_highest(lst):
  best = 0
  for d in lst:
    curr = int(math.ceil(d[1].max() / 10.0) * 10)
    if curr>best:
      best = curr
  return best

def compare_5(df, df2, df3, df4, df5):
  #get the upper limit and round up to the nearest 10
  upper_limit = get_highest([df, df2, df3, df4, df5])


  chart1 = alt.Chart(df).mark_bar().encode(
      x=alt.X('1:Q', title='Average count for Tech', scale=alt.Scale(domain=[0, upper_limit])),
      y=alt.Y("0:O", title = 'POS', sort='y')
  )

  chart2 = alt.Chart(df2).mark_bar().encode(
      x=alt.X('1:Q', title='Average count for Sports', scale=alt.Scale(domain=[0, upper_limit])),
      y=alt.Y("0:O", title='', sort='y')
  )

  chart3 = alt.Chart(df3).mark_bar().encode(
      x=alt.X('1:Q', title='Average count for Politics', scale=alt.Scale(domain=[0, upper_limit])),
       y=alt.Y("0:O", title='', sort='y')
  )

  chart4 = alt.Chart(df4).mark_bar().encode(
      x=alt.X('1:Q', title='Average count for Entertainment', scale=alt.Scale(domain=[0, upper_limit])),
      y=alt.Y("0:O", title='', sort='y')
  )

  chart5 = alt.Chart(df5).mark_bar().encode(
      x=alt.X('1:Q', title='Average count for Business', scale=alt.Scale(domain=[0, upper_limit])),
      y=alt.Y("0:O", title='', sort='y')
  )

  return chart1 | chart2 | chart3 | chart4 | chart5

compare_5(df, df2, df3, df4, df5)

alt.HConcatChart(...)

In [ ]:

#after, average them for each category and test plot on histogram to see if there is a difference (for both Summaries & articles)
#finally, look at distribution compared against same category for articles & Summaries


